# Revision class
## Use various data to create our very own database
### This exercise will compromises all of the courses learnt throughout Data Analyst in SQL
### For this exercise, we will be using Bicycle Sales database
### Refer here [](https://www.sqlservertutorial.net/sql-server-sample-database/)

In [2]:
/* BICYCLE SALES DATABASE */


-- Let's take a brief overview of the schemas in the Bicycle Sales databases
-- Production schema

SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'production'



In [3]:
-- Stocks schema

SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'sales'

In [4]:
-- Select product name, brand name, category, list price, model year and 

SELECT product_name, category_name, list_price, model_year, SUM(quantity) AS stocks_count
FROM production.categories AS c
INNER JOIN production.products AS p
ON c.category_id = p.category_id
INNER JOIN production.stocks AS s
ON p.product_id = s.product_id
GROUP BY product_name, category_name, list_price, model_year

-- Turns out that Electra Townie Commute GO! & Electra Moto 3 are currently in stock with 75 items are in stock each (highest)
-- Also, there are usually the most expensive item in stocks ($ 2,999.99)

In [5]:
-- How many stores are there listed in the stocks table

SELECT store_id, COUNT(store_id) AS stores_count
FROM production.stocks
GROUP BY store_id;

-- Currently, there are 3 stores id and total is 939 stores

In [6]:
-- How many items are in stock in the stock table?

SELECT store_id, SUM(quantity) AS total_stocks
FROM production.stocks
GROUP BY store_id;

-- Store id 3 has the highest total number of items in stock

In [7]:
-- Looking at the schema, in the staffs table manager_id and staff_id have many to one relationship
-- But how true is this? Let's find out

SELECT manager_id, COUNT(staff_id) AS staff_count
FROM sales.staffs
GROUP BY manager_id

-- There is one staff that currently have no managers attached to FROM

In [8]:
-- Let's find out how many products that do not have its joined counterparts in the stocks table

SELECT p.product_id, p.product_name
FROM production.products AS p
LEFT JOIN production.stocks AS st
ON p.product_id = st.product_id
WHERE st.product_id IS NULL;

-- If we left join table products with table stocks, the return will be 947 items
-- However, about 8 products whose product id is not found in the stocks table

In [9]:
-- Let's check the stores table from the sales schema
-- This time, we would like to see the store_id's name

SELECT s.store_id, s.store_name, st.quantity AS stocks, s.street, s.city, s.state 
FROM production.stocks AS st
RIGHT JOIN sales.stores AS s
ON st.store_id = s.store_id;

In [10]:
-- How many cities or states are there in the sales table?

SELECT store_id, city, state, COUNT(*) AS count
FROM sales.stores
GROUP BY store_id, city, state;

-- Currently, the bike shops are located in 3 cities, each cities reside in 3 different states
-- It is also known that store_id = 1 refers to CA state, 2 refers to NY and 3 refers to TX

In [11]:
-- How many brands are there

SELECT COUNT(*) AS no_of_brands
FROM production.brands

-- There are a total of 9 brands

In [12]:
-- Find out which products have their total stocks to be less than the total average number of items in stock

SELECT p.product_id, p.product_name, p.brand_id, p.list_price
FROM production.products AS p
LEFT JOIN production.stocks AS st
ON p.product_id = st.product_id
WHERE st.product_id IS NOT NULL
GROUP BY p.product_id, p.product_name, p.brand_id, p.list_price
HAVING (SELECT AVG(st2.quantity)
	   FROM production.stocks AS st2) > (
		SELECT SUM(st3.quantity)
		FROM production.stocks AS st3
		WHERE st3.product_id = p.product_id)
		
-- There were only 9 products whose total number in stocks is less than the total average

In [13]:
SELECT *
FROM production.products AS p1
WHERE p1.category_id = 6

In [14]:
-- Let's find out which products whose list price is equal to the highest list price of the products within the same category:

SELECT p1.product_id, p1.product_name, p1.list_price, p1.category_id
FROM production.products AS p1
WHERE p1.list_price IN (SELECT MAX(p2.list_price)
					   FROM production.products AS p2
					   WHERE p2.category_id = p1.category_id
					   GROUP BY p2.category_id)
					   
-- Now, it's easier to see which products whose price is the highest for its categories
-- The brand Trek seems to dominate the list, second next is Electra 

In [15]:
-- Let's find out all of the product's listing prices and what is their product counterpart most expensive items for each respective brand
-- How big is the difference between their listing prices?

SELECT p.product_name, b.brand_name, p.list_price, (SELECT MAX(p1.list_price)
										FROM production.stocks AS st1
									  	INNER JOIN production.products AS p1
									  	ON st1.product_id = p1.product_id
									  	WHERE st1.store_id = st.store_id
										AND p1.brand_id = b.brand_id
									 	) AS max_price, 
										
										(p.list_price - (SELECT 		MAX(p1.list_price)
										FROM production.stocks AS st1
									  	INNER JOIN production.products AS p1
									  	ON st1.product_id = p1.product_id
									  	WHERE st1.store_id = st.store_id
										AND p1.brand_id = b.brand_id
									 	)) AS diff, 
		st.quantity, s.city, s.state
FROM production.products AS p
LEFT JOIN production.stocks AS st
ON p.product_id = st.product_id
LEFT JOIN sales.stores AS s
ON st.store_id = s.store_id
LEFT JOIN production.brands AS b
ON p.brand_id = b.brand_id
WHERE st.product_id IS NOT NULL
GROUP BY p.product_name, p.list_price, st.quantity, s.city, s.state, st.store_id, b.brand_id, b.brand_name;

-- As expected, Electra came out on top 

In [16]:
-- Find out what categories are there for all bikes products?

SELECT category_id, category_name, COUNT(*) AS count
FROM production.categories
GROUP BY category_id, category_name;

-- We have 7 types categories for all bike products

In [17]:
-- Find out which brands has the highest number of product line

SELECT b.brand_name, c.category_name, COUNT(product_id) AS count
FROM production.products AS p
LEFT JOIN production.categories AS c
ON p.category_id = c.category_id
INNER JOIN production.brands as b
ON p.brand_id = b.brand_id
GROUP BY ROLLUP (b.brand_name, c.category_name)

-- Turns out that the brand Electra has the highest number for the category Cruisters Bicycles, followed by Trek for Road Bikes and Mountain Bikes
-- By sorting out the category name, Trek, Electra and Surly have number for all combined categories

In [18]:
-- Let's find out the total inventories or stock levels based on the categories, brands and the which state does keep track of their products

SELECT s.state, c.category_name, b.brand_name, SUM(pc.st_quantity) AS total_stock, MIN(pc.p_list_price) AS price
FROM production.categories AS c
RIGHT JOIN (SELECT p.category_id AS p_category_id, p.brand_id AS p_brand_id, st.store_id AS st_store_id, st.quantity AS st_quantity, p.list_price AS p_list_price
		   FROM production.products AS p
		   LEFT JOIN production.stocks AS st
		   ON p.product_id = st.product_id) AS pc
ON pc.p_category_id = c.category_id
LEFT JOIN sales.stores AS s 
ON pc.st_store_id = s.store_id
LEFT JOIN production.brands AS b
ON pc.p_brand_id = b.brand_id
GROUP BY s.state, c.category_name, b.brand_name;

-- Cruiser Bicycles category under the brand Electra are commonly found across all the three states, priced at $269.99 each
-- Followed by Road Bikes under the brand Trek
-- What is interesting to see is that note all categories of the same brand are distributed evenly across the states, for example, Trek is commonly found in Texas for Children Bicycles whereas Electra are more common in California and New York states
-- Unfortuntely, we have also found that some of the brands such as Electra and Trek does not even have product_id assigned


In [19]:
-- Let's find out which brands that does not have its own product ids?

SELECT p.product_name, p.brand_id, p.category_id, p.model_year, p.list_price
FROM production.products AS p
LEFT JOIN production.stocks AS st 
ON p.product_id = st.product_id
WHERE st.product_id IS NULL

-- There were a total of 8 products that do not have any product ids assigned
-- Without it, it could prove difficult to keep track of the inventories

In [20]:
-- Let's take a look how orders have been made for the bicycles
-- First of all, how many orders were made to order these bicycles and how much of revenue can be generated (before and after discount)?

SELECT SUM(oi.quantity) AS no_of_itmes_ordered, SUM(oi.quantity * oi.list_price) AS total_sales_wo_discount, SUM(oi.quantity * oi.list_price * (1-oi.discount)) AS total_sales_after_discount
FROM sales.order_items AS oi
RIGHT JOIN production.products AS p
ON oi.product_id = p.product_id

-- That is an impressive total of 7,078, $ 8.5 million potential revenue (before discount) and $ 7.6 million potential revenue (after discount)

In [21]:
-- Let's now find out products total sales after discount for each product line and each of their categories

SELECT p.product_id AS ProductID, p.product_name AS Product, b.brand_name AS Brand, c.category_name AS Category, SUM(oi.quantity) AS Quantity, SUM(oi.quantity * oi.list_price * (1-oi.discount)) AS total_sales
FROM sales.order_items AS oi
LEFT JOIN production.products AS p
ON oi.product_id = p.product_id
LEFT JOIN production.brands as b
ON p.brand_id = b.brand_id
LEFT JOIN production.categories As c
ON p.category_id = c.category_id
WHERE p.brand_id IN 
	 	(SELECT b1.brand_id
		FROM production.brands AS b1)
GROUP BY p.product_id, p.product_name, b.brand_name, c.category_name;

-- Measures such as Quantities refers to the orders made to number of items that the customers want to purchase
-- Surly appeared as number 1 in the most ordered items (Surly Ice Cream Truck) where the number of bikes ordered is 167 items, followed by Electra products
-- Trek is one of the best and popular products lines, next to Surly when it comes to total sales (with the highest recorded amount is $555,558) 


,ProductID,Product,Brand,Category,Quantity,total_sales
0,2,Ritchey Timberwolf Frameset - 2016,Ritchey,Mountain Bikes,118,78898.9480
1,3,Surly Wednesday Frameset - 2016,Surly,Mountain Bikes,126,112288.8771
2,4,Trek Fuel EX 8 29 - 2016,Trek,Mountain Bikes,143,368472.7294
3,5,Heller Shagamaw Frame - 2016,Heller,Mountain Bikes,129,151160.8857
4,6,Surly Ice Cream Truck Frameset - 2016,Surly,Mountain Bikes,167,70371.6027
...,...,...,...,...,...,...
302,311,Electra Townie Commute 8D - 2018,Electra,Comfort Bicycles,3,1874.9750
303,312,Electra Townie Commute 8D Ladies' - 2018,Electra,Comfort Bicycles,2,1259.9820
304,313,Electra Townie Original 1 Ladies' - 2018,Electra,Comfort Bicycles,4,1574.9650
305,314,Electra Townie Original 21D EQ Ladies' - 2018,Electra,Comfort Bicycles,2,1087.9840


In [22]:
-- Let's now find out products total sales after discount, how each products fares against the total sales within each category

SELECT oi.product_id AS ProductID, pb.p_product_name AS Product, pb.b_brand_name AS Brand, c.category_name AS Category ,SUM(oi.quantity * oi.list_price * (1-oi.discount)) AS total_sales, 
	
	(SELECT SUM(oi1.quantity * oi1.list_price * (1-oi1.discount))
	 FROM sales.order_items AS oi1
	 LEFT JOIN production.products AS p1
	 ON oi1.product_id = p1.product_id
	 WHERE p1.category_id = pb.p_category_id
	GROUP BY p1.category_id) AS category_total_sales
	
FROM sales.order_items AS oi
LEFT JOIN
	(SELECT p.product_id AS p_product_id, p.product_name AS p_product_name, b.brand_id AS b_brand_id, b.brand_name AS b_brand_name, p.category_id AS p_category_id
	FROM production.products AS p
	LEFT JOIN production.brands AS b
	ON p.brand_id = b.brand_id) AS pb
ON oi.product_id = pb.p_product_id
LEFT JOIN production.categories AS c
ON pb.p_category_id = c.category_id
GROUP BY oi.product_id, pb.p_product_name, pb.p_category_id, pb.b_brand_name, c.category_name

-- Mountain Bikes, Road Bikes and Cruiser Bikes are really popular among the customers
-- As for which brand records the highest sales after discount, Trek tops for the Mountain and Road Bikes category
-- The only exception is the Electra in the Cruisers categories, which records their biggest total sales compared to their other products lines in the rest of the categories

,ProductID,Product,Brand,Category,total_sales,category_total_sales
0,21,Electra Cruiser 1 (24-Inch) - 2016,Electra,Children Bicycles,34078.1378,2.921892e+05
1,22,Electra Girl's Hawaii 1 (16-inch) - 2015/2016,Electra,Children Bicycles,34728.8137,2.921892e+05
2,23,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,Electra,Children Bicycles,41011.6329,2.921892e+05
3,83,Trek Boy's Kickster - 2015/2017,Trek,Children Bicycles,3236.7842,2.921892e+05
4,84,Sun Bicycles Lil Kitt'n - 2017,Sun Bicycles,Children Bicycles,2328.4883,2.921892e+05
...,...,...,...,...,...,...
302,180,Trek Domane AL 3 Women's - 2018,Trek,Road Bikes,6219.1324,1.665098e+06
303,181,Trek Domane SL 5 - 2018,Trek,Road Bikes,10031.9544,1.665098e+06
304,182,Trek Domane SL 5 Disc - 2018,Trek,Road Bikes,11074.9557,1.665098e+06
305,183,Trek Domane SL 5 Women's - 2018,Trek,Road Bikes,1759.9920,1.665098e+06


In [23]:


-- Now that we have obtained sales for each products of their own category as well as against total category, combine the two sets of table as one
-- 

WITH brand_for_category_sales AS (
	SELECT p.product_id AS ProductID, p.product_name AS Product, b.brand_name AS Brand, c.category_name AS Category, SUM(oi.quantity) AS Quantity, SUM(oi.quantity * oi.list_price * (1-oi.discount)) AS total_sales
	FROM sales.order_items AS oi
	LEFT JOIN production.products AS p
	ON oi.product_id = p.product_id
	LEFT JOIN production.brands as b
	ON p.brand_id = b.brand_id
	LEFT JOIN production.categories As c
	ON p.category_id = c.category_id
	WHERE p.brand_id IN 
	 	(SELECT b1.brand_id
		FROM production.brands AS b1)
	GROUP BY p.product_id, p.product_name, b.brand_name, c.category_name
),
category_sales AS (
SELECT oi.product_id AS ProductID, pb.p_product_name AS Product, pb.b_brand_name AS Brand, c.category_name AS Category ,SUM(oi.quantity * oi.list_price * (1-oi.discount)) AS total_sales, 
	
	(SELECT SUM(oi1.quantity * oi1.list_price * (1-oi1.discount))
	 FROM sales.order_items AS oi1
	 LEFT JOIN production.products AS p1
	 ON oi1.product_id = p1.product_id
	 WHERE p1.category_id = pb.p_category_id
	GROUP BY p1.category_id) AS category_total_sales
	
FROM sales.order_items AS oi
LEFT JOIN
	(SELECT p.product_id AS p_product_id, p.product_name AS p_product_name, b.brand_id AS b_brand_id, b.brand_name AS b_brand_name, p.category_id AS p_category_id
	FROM production.products AS p
	LEFT JOIN production.brands AS b
	ON p.brand_id = b.brand_id) AS pb
ON oi.product_id = pb.p_product_id
LEFT JOIN production.categories AS c
ON pb.p_category_id = c.category_id
GROUP BY oi.product_id, pb.p_product_name, pb.p_category_id, pb.b_brand_name, c.category_name)

SELECT bc.Product, bc.Brand, bc.Category, SUM(bc.total_sales) AS total_sales_by_brand ,cs.category_total_sales
FROM brand_for_category_sales AS bc
INNER JOIN category_sales as cs
ON bc.ProductID = cs.ProductID
GROUP BY bc.Product, bc.Brand, bc.Category, bc.total_sales, cs.category_total_sales

-- By filtering each categories, we will know which brand dominates for each category in terms of its sales (Children Bicyles, Comfort Bicycles, Cruisers Bicycles, Cyclocross Bicycles, Electric Bikes, Mountain Bikes & Road Bikes)
-- For the Children Bicyles category - Electra dominates
-- For the Comfort Bicycles category - Electra dominates
-- For the Cruisers Bicycles category - Electra dominates
-- For the Cyclocross Bicycles category - Surly dominates
-- For the Electric Bikes category - Trek dominates
-- For the Mountain Bikes category - Trek dominates
-- For the Road Bikes category - Trek dominates


,Product,Brand,Category,total_sales_by_brand,category_total_sales
0,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,Electra,Cruisers Bicycles,4004.9555,9.950326e+05
1,Electra Amsterdam Fashion 7i Ladies' - 2017,Electra,Cruisers Bicycles,39863.6376,9.950326e+05
2,Electra Amsterdam Original 3i - 2015/2017,Electra,Cruisers Bicycles,18987.9123,9.950326e+05
3,Electra Amsterdam Original 3i Ladies' - 2017,Electra,Cruisers Bicycles,17793.3304,9.950326e+05
4,Electra Amsterdam Royal 8i - 2017/2018,Electra,Cruisers Bicycles,1133.9100,9.950326e+05
...,...,...,...,...,...
302,Trek X-Caliber 8 - 2017,Trek,Mountain Bikes,30939.6906,2.715080e+06
303,Trek X-Caliber 8 - 2018,Trek,Mountain Bikes,2499.9750,2.715080e+06
304,Trek X-Caliber Frameset - 2018,Trek,Mountain Bikes,5219.9652,2.715080e+06
305,Trek XM700+ - 2018,Trek,Electric Bikes,24149.9310,9.166848e+05


In [24]:
-- Let's explore the  company's manpower capabilities
-- Find out how many employees and managers for all stores across the States?

SELECT store_id AS StoreID, COUNT(staff_id) AS total_staffs, COUNT(manager_id) AS total_managers
FROM sales.staffs 
GROUP BY store_id

-- So each stores, we can expect there will be 3 staffs and each of the staffs would been assigned to their own managers
-- But for storeID 1, we have an extra of 1 employee...

,StoreID,total_staffs,total_managers
0,1,4,3
1,2,3,3
2,3,3,3


In [25]:
-- Let's find out who is that employee..

SELECT *
FROM sales.staffs

-- Turns out Fabiola Jackson does not have a manager, in fact, it could mean that she must be in the highest heirarchy
-- The easiest assumption would be is that Fabiola might have been the founder or CEO, which the latter would make a lot of sense if we're talking about employees
-- Another guess we can make is that some of the employees here are already assigned as managers
-- The managers are identified is Mireya Copeland, Jannette David & Kali Vargas, where each of them reports to Fabiola Jackson...

In [26]:
-- Find out the employees' workplace (street and state)

SELECT s.staff_id AS staffID, CONCAT(s.first_name, ' ', s.last_name) AS name, ss.store_name AS store_name, s.store_id AS storeID
FROM sales.staffs AS s
LEFT JOIN sales.stores AS ss
ON s.store_id = ss.store_id

,staffID,name,store_name,storeID
0,1,Fabiola Jackson,Santa Cruz Bikes,1
1,2,Mireya Copeland,Santa Cruz Bikes,1
2,3,Genna Serrano,Santa Cruz Bikes,1
3,4,Virgie Wiggins,Santa Cruz Bikes,1
4,5,Jannette David,Baldwin Bikes,2
5,6,Marcelene Boyer,Baldwin Bikes,2
6,7,Venita Daniel,Baldwin Bikes,2
7,8,Kali Vargas,Rowlett Bikes,3
8,9,Layla Terrell,Rowlett Bikes,3
9,10,Bernardine Houston,Rowlett Bikes,3


In [27]:
-- Let's find out how orders are connected with orders_item tables
-- We know that customers who ordered the items from the Bike Company would have their orders information stored
-- But first, we would like combine both the order tables that contain how many orders were placed by the customer and the orders_items which contains the total items purchased by the customer

SELECT o.order_id AS orderID, o.customer_id AS customer, o.order_status AS status, o.staff_id AS staffID, o.store_id AS storeID, SUM(oi.quantity) AS quantity, ROUND(SUM(oi.quantity*oi.list_price*(1-oi.discount)),2) AS total_sales
FROM sales.orders AS o
LEFT JOIN sales.order_items AS oi
ON o.order_id = oi.order_id
GROUP BY o.order_id, o.customer_id, o.order_status, o.staff_ID, o.store_id

,orderID,customer,status,staffID,storeID,quantity,total_sales
0,1,259,4,2,1,8,10231.05
1,2,1212,4,6,2,3,1697.97
2,3,523,4,7,2,2,1519.98
3,4,175,4,3,1,2,1349.98
4,5,1324,4,6,2,4,3900.06
...,...,...,...,...,...,...,...
1610,1611,6,3,7,2,4,8963.96
1611,1612,3,3,3,1,8,3781.13
1612,1613,1,3,6,2,3,5257.97
1613,1614,135,3,8,3,5,6104.04


In [28]:
-- Create a CTE and use CTE to reference which employees that collects the most orders from their customers

WITH bike_orders AS (
	SELECT o.order_id AS orderID, o.customer_id AS customer, o.order_status AS status, o.staff_id AS staffID, o.store_id AS storeID, SUM(oi.quantity) AS quantity, ROUND(SUM(oi.quantity*oi.list_price*(1-oi.discount)),2) AS total_sales
	FROM sales.orders AS o
	LEFT JOIN sales.order_items AS oi
	ON o.order_id = oi.order_id
	GROUP BY o.order_id, o.customer_id, o.order_status, o.staff_ID, o.store_id),
	
employee AS (
	SELECT s.staff_id AS staffID, CONCAT(s.first_name, ' ', s.last_name) AS name, ss.store_name AS store_name, s.store_id AS storeID, ss.state AS state
	FROM sales.staffs AS s
	LEFT JOIN sales.stores AS ss
	ON s.store_id = ss.store_id)

SELECT employee.name AS employee_name, employee.store_name AS bike_store, employee.state AS bike_state, SUM(bike_orders.total_sales) AS bike_sales, COUNT(bike_orders.customer) AS no_of_customers
FROM employee
LEFT JOIN bike_orders
ON employee.staffID = bike_orders.staffID
GROUP BY employee.name, employee.store_name, employee.state
ORDER BY bike_sales DESC;

-- Turns out that Baldwin Bikes from NY sold the most bikes, where Marcelen had brought in a total of $2,624,120.33 from 553 customers, followed by Venita who was closing in Marcelene at $2,591,630.3 from 540 customers
-- Besides Fabiola and Janeette are two most senior or upper executive who did not bring any sales, Virgie and Bernadine did not manage to bring sales to the Bike Company

,employee_name,bike_store,bike_state,bike_sales,no_of_customers
0,Marcelene Boyer,Baldwin Bikes,NY,2624120.33,553
1,Venita Daniel,Baldwin Bikes,NY,2591630.30,540
2,Genna Serrano,Santa Cruz Bikes,CA,853287.20,184
3,Mireya Copeland,Santa Cruz Bikes,CA,752535.55,164
4,Kali Vargas,Rowlett Bikes,TX,463918.31,88
5,Layla Terrell,Rowlett Bikes,TX,403623.85,86
6,Jannette David,Baldwin Bikes,NY,NaN,0
7,Virgie Wiggins,Santa Cruz Bikes,CA,NaN,0
8,Fabiola Jackson,Santa Cruz Bikes,CA,NaN,0
9,Bernardine Houston,Rowlett Bikes,TX,NaN,0


In [29]:
-- Let's compare their sales against the average sales of the stores

WITH bike_orders AS (
	SELECT o.order_id AS orderID, o.customer_id AS customer, o.order_status AS status, o.staff_id AS staffID, o.store_id AS storeID, SUM(oi.quantity) AS quantity, ROUND(SUM(oi.quantity*oi.list_price*(1-oi.discount)),2) AS total_sales
	FROM sales.orders AS o
	LEFT JOIN sales.order_items AS oi
	ON o.order_id = oi.order_id
	GROUP BY o.order_id, o.customer_id, o.order_status, o.staff_ID, o.store_id),
	
employee AS (
	SELECT s.staff_id AS staffID, CONCAT(s.first_name, ' ', s.last_name) AS name, ss.store_name AS store_name, s.store_id AS storeID, ss.state AS state
	FROM sales.staffs AS s
	LEFT JOIN sales.stores AS ss
	ON s.store_id = ss.store_id)

SELECT employee.name AS employee_name, employee.store_name AS bike_store, employee.state AS bike_state, SUM(bike_orders.total_sales) AS bike_sales,
 AVG(SUM(bike_orders.total_sales)) OVER (PARTITION BY employee.state) AS avg_bike_sales_by_state
FROM employee
LEFT JOIN bike_orders
ON employee.staffID = bike_orders.staffID
GROUP BY employee.name, employee.store_name, employee.state
ORDER BY bike_sales DESC;

,employee_name,bike_store,bike_state,bike_sales,avg_bike_sales_by_state
0,Marcelene Boyer,Baldwin Bikes,NY,2624120.33,2607875.315
1,Venita Daniel,Baldwin Bikes,NY,2591630.30,2607875.315
2,Genna Serrano,Santa Cruz Bikes,CA,853287.20,802911.375
3,Mireya Copeland,Santa Cruz Bikes,CA,752535.55,802911.375
4,Kali Vargas,Rowlett Bikes,TX,463918.31,433771.080
5,Layla Terrell,Rowlett Bikes,TX,403623.85,433771.080
6,Bernardine Houston,Rowlett Bikes,TX,NaN,433771.080
7,Virgie Wiggins,Santa Cruz Bikes,CA,NaN,802911.375
8,Jannette David,Baldwin Bikes,NY,NaN,2607875.315
9,Fabiola Jackson,Santa Cruz Bikes,CA,NaN,802911.375


In [30]:
-- Let's find out more about the customers of the Bike Stores
-- First, we need to find out how many bikes were usually sold to the customers
-- And, we need  to find out how much of these customers spent to get their orders

SELECT so.customer_id AS customerID, CONCAT(sc.first_name, ' ', sc.last_name) AS customer, COUNT(so.order_id) AS no_of_orders, ROUND(SUM(soi.quantity*soi.list_price*(1-soi.discount)),2) AS money_spent, sc.city, sc.state
FROM sales.customers AS sc
RIGHT JOIN sales.orders AS so
ON sc.customer_id = so.customer_id
LEFT JOIN sales.order_items AS soi 
ON soi.order_id = so.order_id
GROUP BY so.customer_id, CONCAT(sc.first_name, ' ', sc.last_name), sc.city, sc.state
ORDER BY money_spent DESC;

-- Sharyn from Baldwinsville has spent a cumulative of $34,807.94 where she made a total of 10 orders from the bike store
-- Besides Sharyn, Pameila or Abby are also spent a considerable amount of cash for the bikes
-- It seems that most of the orders that made into the top 10 were from the state of NY, but how sure are we?

,customerID,customer,no_of_orders,money_spent,city,state
0,94,Sharyn Hopkins,10,34807.94,Baldwinsville,NY
1,10,Pamelia Newman,11,33634.26,Monroe,NY
2,75,Abby Gamble,7,32803.01,Amityville,NY
3,6,Lyndsey Bean,11,32675.07,Fairport,NY
4,16,Emmitt Sanchez,12,31925.89,New York,NY
...,...,...,...,...,...,...
1440,177,Carissa Foreman,1,170.99,Maspeth,NY
1441,752,Lloyd Miranda,1,170.99,Encino,CA
1442,1111,Caroline Jenkins,1,170.99,Richardson,TX
1443,718,Luciana Mcgee,1,167.99,Selden,NY


In [31]:
-- Using rollups to find out the cummulative sales for all cities and how much total sales were made for each state?

SELECT  COUNT(so.order_id) AS no_of_orders, ROUND(SUM(soi.quantity*soi.list_price*(1-soi.discount)),2) AS money_spent, sc.city, sc.state
FROM sales.customers AS sc
RIGHT JOIN sales.orders AS so
ON sc.customer_id = so.customer_id
LEFT JOIN sales.order_items AS soi 
ON soi.order_id = so.order_id
WHERE so.customer_id IS NOT NULL
GROUP BY ROLLUP (sc.state, sc.city)
ORDER BY money_spent DESC;

-- NY tops the list, followed by CA and TX
-- As expected, customers from Mount Vernon had the most orders and money spent ($105,563.33), followed by customers from Ballston Spa from NY
-- The third biggest sales belongs to San Angelo in the TX state ($98,429.26)

,no_of_orders,money_spent,city,state
0,4722,7689116.56,None,None
1,3195,5215751.28,None,NY
2,1006,1605823.04,None,CA
3,521,867542.24,None,TX
4,60,105563.33,Mount Vernon,NY
...,...,...,...,...
194,4,3089.58,Copperas Cove,TX
195,5,2957.94,Far Rockaway,NY
196,3,1761.15,Springfield Gardens,NY
197,2,1256.09,Tonawanda,NY


In [32]:
-- Let's dig deeper to understand these customers
-- Rank the customers according to the number of orders made

SELECT DENSE_RANK() OVER (PARTITION BY sc.state ORDER BY COUNT(so.order_id) DESC) AS ranking, CONCAT(sc.first_name, ' ', sc.last_name) AS customer, COUNT(so.order_id) AS no_of_orders, sc.city, sc.state
FROM sales.customers AS sc
RIGHT JOIN sales.orders AS so
ON sc.customer_id = so.customer_id
LEFT JOIN sales.order_items AS soi 
ON soi.order_id = so.order_id
GROUP BY so.customer_id, CONCAT(sc.first_name, ' ', sc.last_name), sc.city, sc.state

-- Toggling between the three states, we can find that Tameka Fisher from Redondo Beach of CA, Emmit Sanchez from New York of NY & Mozelle Carter from Houstan of TX ranks number 1 within their own states

,ranking,customer,no_of_orders,city,state
0,1,Tameka Fisher,13,Redondo Beach,CA
1,2,Corene Wall,11,Atwater,CA
2,3,Jamaal Albert,10,Torrance,CA
3,3,Kasha Todd,10,Campbell,CA
4,3,Ronna Butler,10,Encino,CA
...,...,...,...,...,...
1440,11,Brent Calderon,1,Corpus Christi,TX
1441,11,Carisa Carpenter,1,Victoria,TX
1442,11,Erik Leblanc,1,Sugar Land,TX
1443,11,Caroline Jenkins,1,Richardson,TX


In [33]:
-- Instead of number of orders, what about the amout of money spent for their orders?

SELECT DENSE_RANK() OVER (PARTITION BY sc.state ORDER BY ROUND(SUM(soi.quantity*soi.list_price*(1-soi.discount)),2) DESC) AS ranking, CONCAT(sc.first_name, ' ', sc.last_name) AS customer, ROUND(SUM(soi.quantity*soi.list_price*(1-soi.discount)),2) AS money_spent, sc.city, sc.state
FROM sales.customers AS sc
RIGHT JOIN sales.orders AS so
ON sc.customer_id = so.customer_id
LEFT JOIN sales.order_items AS soi 
ON soi.order_id = so.order_id
GROUP BY so.customer_id, CONCAT(sc.first_name, ' ', sc.last_name), sc.city, sc.state

-- Toggling between the three states, we can find that Tameka Fisher from Redondo Beach of CA, Sharyn Hopkins from Baldwinsville of NY & Elinore Aguilar from San Angelo of TX ranks number 1 within their own states
-- Tameka Fisther, till today, remains as the only customer that spends highest amount of money together with the one with the highest orders made (orders = 13, $24,051.53) in her state of NY

,ranking,customer,money_spent,city,state
0,1,Tameka Fisher,24051.53,Redondo Beach,CA
1,2,Brigid Sharp,20648.95,Santa Clara,CA
2,3,Williemae Holloway,19756.64,Oakland,CA
3,4,Kasha Todd,19329.08,Campbell,CA
4,5,Deloris Burke,19043.90,Palos Verdes Peninsula,CA
...,...,...,...,...,...
1440,138,Lilliam Nolan,242.99,Copperas Cove,TX
1441,138,Launa Hull,242.99,Helotes,TX
1442,139,Clementine Mooney,199.99,Mcallen,TX
1443,140,Caroline Jenkins,170.99,Richardson,TX


In [34]:
-- Let's study about the movement of these items and the progress of the deliveries
-- Use moving averages to track the number of items shipped to the destination

SELECT CONCAT(YEAR(so.shipped_date),'-',MONTH(so.shipped_date),'-',DAY(so.shipped_date)) AS Shipped_Date, COUNT(soi.order_id) AS no_of_orders, 

	AVG(CAST(COUNT(soi.order_id) AS DECIMAL)) OVER(ORDER BY so.shipped_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS [7_days_avg_movement],
	AVG(CAST(COUNT(soi.order_id) AS DECIMAL)) OVER(ORDER BY so.shipped_date ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) AS [14_days_avg_movement],
	AVG(CAST(COUNT(soi.order_id) AS DECIMAL)) OVER(ORDER BY so.shipped_date ROWS BETWEEN 20 PRECEDING AND CURRENT ROW) AS [21_days_avg_movement]

FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
LEFT JOIN sales.customers AS sc
ON so.customer_id = sc.customer_id
WHERE so.shipped_date IS NOT NULL
GROUP BY so.shipped_date
ORDER BY so.shipped_date ASC;

,Shipped_Date,no_of_orders,7_days_avg_movement,14_days_avg_movement,21_days_avg_movement
0,2016-01-03,9,9.000000,9.000000,9.000000
1,2016-01-05,11,10.000000,10.000000,10.000000
2,2016-01-06,4,8.000000,8.000000,8.000000
3,2016-01-07,3,6.750000,6.750000,6.750000
4,2016-01-08,1,5.600000,5.600000,5.600000
...,...,...,...,...,...
670,2018-03-29,8,8.857142,8.285714,7.714285
671,2018-03-30,4,8.428571,8.500000,7.714285
672,2018-03-31,9,8.142857,8.785714,7.904761
673,2018-04-01,6,6.142857,7.785714,7.523809


In [35]:
-- Let's find out about the deliveries made within the schedule
-- Calculate the number of days that the bike companies takes to ship their products upon orders made and against their delivery target date


SELECT shipped_date, required_date, COUNT(soi.order_id) AS no_of_orders,
	CASE WHEN shipped_date <= required_date THEN DATEDIFF(day, shipped_date, required_date) 
	WHEN shipped_date >= required_date THEN DATEDIFF(day, shipped_date, required_date) ELSE NULL END AS days_to_deliver	
FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
GROUP BY shipped_date, required_date

-- Now it is easier to see how the orders were moved to its destination by looking at the differences between the date that it ships against the required date to it to be delivered to the customers

,shipped_date,required_date,no_of_orders,days_to_deliver
0,2016-02-11 00:00:00+00:00,2016-02-12 00:00:00+00:00,3,1.0
1,2016-03-23 00:00:00+00:00,2016-03-24 00:00:00+00:00,2,1.0
2,2016-05-03 00:00:00+00:00,2016-05-04 00:00:00+00:00,5,1.0
3,2017-02-17 00:00:00+00:00,2017-02-17 00:00:00+00:00,3,0.0
4,2017-06-20 00:00:00+00:00,2017-06-20 00:00:00+00:00,6,0.0
...,...,...,...,...
1256,2016-09-19 00:00:00+00:00,2016-09-19 00:00:00+00:00,4,0.0
1257,2016-10-07 00:00:00+00:00,2016-10-08 00:00:00+00:00,2,1.0
1258,2016-11-01 00:00:00+00:00,2016-11-01 00:00:00+00:00,4,0.0
1259,2016-12-07 00:00:00+00:00,2016-12-09 00:00:00+00:00,5,2.0


In [36]:
-- Find out the total n orders made within the schedule and the orders that were delivered late

WITH orders_shipped AS ( 
	SELECT 
	COUNT(CASE WHEN shipped_date <= required_date THEN soi.order_id ELSE NULL END) AS on_time_delivery, 
	COUNT(CASE WHEN shipped_date >= required_date THEN soi.order_id ELSE NULL END) AS late_delivery	
FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
GROUP BY shipped_date, required_date)

SELECT CAST(SUM(on_time_delivery) AS DECIMAL)/(CAST(SUM(on_time_delivery)AS DECIMAL)+CAST(SUM(late_delivery) AS DECIMAL)) AS successful_deliveries
FROM orders_shipped

-- About 50.73% deliveries have been successful with delivering within the schedule
-- On the other hand, another half of the deliveries did not managed to be delivered on time

,successful_deliveries
0,0.507354


In [37]:
-- Let's dive deeper into the findings that we have found
-- Calculate the delivery rate for all deliveries


SELECT shipped_date, required_date, COUNT(soi.order_id) AS no_of_orders,
	CASE WHEN shipped_date IS NULL THEN 0
	WHEN DATEDIFF(day, shipped_date, required_date) = 0 THEN COUNT(soi.order_id)/1.0 
	WHEN DATEDIFF(day, shipped_date, required_date) > 0 THEN COUNT(soi.order_id)/CAST(ABS(DATEDIFF(day, shipped_date, required_date)) AS DECIMAL)
	WHEN DATEDIFF(day, shipped_date, required_date) < 0 THEN COUNT(soi.order_id)/CAST(DATEDIFF(day, shipped_date, required_date) AS DECIMAL)
	ELSE NULL END AS 'Delivery Rate Per Day'
FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
GROUP BY shipped_date, required_date

,shipped_date,required_date,no_of_orders,Delivery Rate Per Day
0,2016-02-11 00:00:00+00:00,2016-02-12 00:00:00+00:00,3,3.0
1,2016-03-23 00:00:00+00:00,2016-03-24 00:00:00+00:00,2,2.0
2,2016-05-03 00:00:00+00:00,2016-05-04 00:00:00+00:00,5,5.0
3,2017-02-17 00:00:00+00:00,2017-02-17 00:00:00+00:00,3,3.0
4,2017-06-20 00:00:00+00:00,2017-06-20 00:00:00+00:00,6,6.0
...,...,...,...,...
1256,2016-09-19 00:00:00+00:00,2016-09-19 00:00:00+00:00,4,4.0
1257,2016-10-07 00:00:00+00:00,2016-10-08 00:00:00+00:00,2,2.0
1258,2016-11-01 00:00:00+00:00,2016-11-01 00:00:00+00:00,4,4.0
1259,2016-12-07 00:00:00+00:00,2016-12-09 00:00:00+00:00,5,2.5


In [38]:
-- Let assess the prior delivery rates to gauge whether the deliveries are efficiently handled from day-to-day businesses


WITH orders_shipped AS ( 
	SELECT shipped_date, required_date, COUNT(soi.order_id) AS no_of_orders,
	CASE WHEN shipped_date IS NULL THEN 0
	WHEN DATEDIFF(day, shipped_date, required_date) = 0 THEN COUNT(soi.order_id)/1.0 
	WHEN DATEDIFF(day, shipped_date, required_date) > 0 THEN COUNT(soi.order_id)/CAST(ABS(DATEDIFF(day, shipped_date, required_date)) AS DECIMAL)
	WHEN DATEDIFF(day, shipped_date, required_date) < 0 THEN COUNT(soi.order_id)/CAST(DATEDIFF(day, shipped_date, required_date) AS DECIMAL)
	ELSE NULL END AS delivery_rate_per_day
FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
GROUP BY shipped_date, required_date
)

SELECT shipped_date, required_date, no_of_orders, delivery_rate_per_day AS 'Delivery Rate Per Day',
LAG(delivery_rate_per_day,1) OVER (ORDER BY required_date) AS 'Previous Delivery Rate Per Day'
FROM orders_shipped
ORDER BY required_date ASC

-- Now we have a better picture on how deliveries are made today and yesterday's

,shipped_date,required_date,no_of_orders,Delivery Rate Per Day,Previous Delivery Rate Per Day
0,2016-01-03 00:00:00+00:00,2016-01-03 00:00:00+00:00,5,5.0,NaN
1,2016-01-03 00:00:00+00:00,2016-01-04 00:00:00+00:00,2,2.0,5.0
2,2016-01-05 00:00:00+00:00,2016-01-04 00:00:00+00:00,1,-1.0,2.0
3,2016-01-03 00:00:00+00:00,2016-01-05 00:00:00+00:00,2,1.0,-1.0
4,2016-01-05 00:00:00+00:00,2016-01-05 00:00:00+00:00,2,2.0,1.0
...,...,...,...,...,...
1256,NaT,2018-09-06 00:00:00+00:00,3,0.0,0.0
1257,NaT,2018-10-21 00:00:00+00:00,5,0.0,0.0
1258,NaT,2018-11-18 00:00:00+00:00,2,0.0,0.0
1259,NaT,2018-11-28 00:00:00+00:00,3,0.0,0.0


In [39]:
-- Next, we include running total for the number of orders made for all deliveries

WITH orders_shipped AS ( 
	SELECT shipped_date, required_date, COUNT(soi.order_id) AS no_of_orders,
	CASE WHEN shipped_date IS NULL THEN 0
	WHEN DATEDIFF(day, shipped_date, required_date) = 0 THEN COUNT(soi.order_id)/1.0 
	WHEN DATEDIFF(day, shipped_date, required_date) > 0 THEN COUNT(soi.order_id)/CAST(ABS(DATEDIFF(day, shipped_date, required_date)) AS DECIMAL)
	WHEN DATEDIFF(day, shipped_date, required_date) < 0 THEN COUNT(soi.order_id)/CAST(DATEDIFF(day, shipped_date, required_date) AS DECIMAL)
	ELSE NULL END AS delivery_rate_per_day
FROM sales.orders AS so
LEFT JOIN sales.order_items AS soi 
ON so.order_id = soi.order_id
GROUP BY shipped_date, required_date
)

SELECT shipped_date, required_date, no_of_orders, delivery_rate_per_day AS 'Delivery Rate Per Day',
SUM(no_of_orders) OVER (ORDER BY required_date ASC) AS 'Running Total for the Deliveries'
FROM orders_shipped
WHERE shipped_date IS NOT NULL
ORDER BY required_date ASC;

-- Discounting shipped_date when it is missing, the Shipped bikes have delivered a total of 4,214 orders to the customers

,shipped_date,required_date,no_of_orders,Delivery Rate Per Day,Running Total for the Deliveries
0,2016-01-03 00:00:00+00:00,2016-01-03 00:00:00+00:00,5,5.0,5
1,2016-01-03 00:00:00+00:00,2016-01-04 00:00:00+00:00,2,2.0,8
2,2016-01-05 00:00:00+00:00,2016-01-04 00:00:00+00:00,1,-1.0,8
3,2016-01-03 00:00:00+00:00,2016-01-05 00:00:00+00:00,2,1.0,12
4,2016-01-05 00:00:00+00:00,2016-01-05 00:00:00+00:00,2,2.0,12
...,...,...,...,...,...
1182,2018-03-31 00:00:00+00:00,2018-04-01 00:00:00+00:00,4,4.0,4211
1183,2018-04-01 00:00:00+00:00,2018-04-01 00:00:00+00:00,5,5.0,4211
1184,2018-04-02 00:00:00+00:00,2018-04-01 00:00:00+00:00,4,-4.0,4211
1185,2018-04-02 00:00:00+00:00,2018-04-02 00:00:00+00:00,2,2.0,4213
